# Main analysis

The primary analysis for the thesis, where we train a classifier for the code vs prose task.

---

### Table of Contents
    
 - [Setup](#Setup)
   - Imports
   - Configuration
 - [Loading](#Loading)
   - Loading EEG data
   - Loading markers
 - [Preprocessing](#Preprocessing)
   - Filter short trials
   - Filter no answer trials
   - Bandpass filtering
   - Constructing epochs
   - Epochs to windows
   - Constructing our `X` and `y`
 - [Training](#Training)
   - Learning curves
      
**NOTE:** This TOC is manually built and may not be up to date.

---

## Setup

In [ ]:
# Imports
import re
import logging
from pathlib import Path
from datetime import datetime, timezone
from typing import Dict
from pprint import pprint
from collections import Counter

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import mne

from eegclassify import main, load, clean, features, preprocess

### Configuration

This cell contains all the configuration options available for the analysis.

In [ ]:
# Configuration
use_bandpass_filter = True
classify_breaks = False
single_subject = None    # set to subject number, or False
balance_dataset = True
min_task_duration = 5
standardize = False
    
# Constants
sfreq = 256  # sampling frequency of the Muse S

### Misc setup

In [ ]:
# Set up logging
logging.basicConfig(format="%(levelname)s: %(msg)s", level=logging.INFO, force=True)
logger = logging.getLogger(__name__)
mne.set_log_level('WARNING')

# Set this to True to run on testing data
simulate_test = False
if simulate_test:
    import os
    os.environ['PYTEST_CURRENT_TEST'] = "true"

In [ ]:
%%javascript
document.title='erb-thesis/Main - Jupyter'  // Set the document title to be able to track time spent working on the notebook with ActivityWatch

## Loading

### Loading EEG

First we need to load the EEG data used during the experiments.

In [ ]:
data_dir = Path('../data').resolve()
    
files = sorted([
    data_dir / "eeg/muse/subject0000/session001/recording_2021-04-02-14.03.36.csv",
    *list((data_dir / "eeg/muse/subject0001/session001/").glob("*.csv")),
])
pprint(files)

eeg = load.load_eeg(files)
eeg = eeg.set_index('timestamp').sort_index()
print(eeg.shape)

Lets have a look at the loaded data:

In [ ]:
def plot_eeg(X, offset, window, span=None):
    plt.figure(figsize=(20, 3))
    data = X[offset : offset + window, :]
    plt.plot(data)
    plt.xlim(0, window);
    if span is None:
        span = max(np.std(data, axis=1))
    plt.ylim(-span, span);

plot_eeg(eeg.to_numpy(), offset = 1000, window = 3 * sfreq)
plot_eeg(eeg.to_numpy(), offset = 20000, window = 3 * sfreq)

We see that some channels are bad some of the time, we will deal with that later.

### Loading markers

Now we need to load the markers produced during each trial of the experiment, so we can annotate the EEG data.

In [ ]:
marker_files = [
    data_dir / 'tasks/visual-codeprose/subject0000/session000/subject0_session0_behOutput_2021-04-02-14.28.30.csv',
    data_dir / 'tasks/visual-codeprose/subject0001/session000/subject1_session0_behOutput_2021-03-26-14.31.14.csv',
]

def _build_breaks(df):
    starts = df['t_answered'].iloc[:-1].shift()
    starts_utc = df['t_answered_utc'].iloc[:-1].shift()
    stops = df['t_presented'].iloc[1:]
    stops_utc = df['t_presented_utc'].iloc[1:]
    
    breaks = pd.DataFrame({
        "t_presented": starts, 
        "t_answered": stops, 
        "t_presented_utc": starts_utc, 
        "t_answered_utc": stops_utc, 
        "type": "relax", 
        "duration": stops - starts, 
        'subject': df['subject'],
        'image_path': 'none',
        'response': 'up',  # as placeholder
    })
    return breaks

dfs = []
for file in marker_files:
    df = pd.read_csv(file, index_col=0)
    df['duration'] = df['t_answered'] - df['t_presented']
    match = re.search('subject(\d+)', str(file))
    assert match
    df['subject'] = int(match.group(1))
    
    if classify_breaks:
        breaks = _build_breaks(df)
        df = df.append(breaks)
    dfs.append(df)
df_markers = pd.concat(dfs).sort_values(by=['subject', 't_presented'])

Lets take a look at some of the marker rows:

In [ ]:
df_markers['img'] = df_markers['image_path'].apply((lambda c: c.split("/")[-1]))
df_markers.drop(columns=['image_path']).head(10)

## Preprocessing

Now we need to preprocess the data a bit, gathering the EEG data for each trial in the experiment.

 - [ ] Better cleaning/rejection of bad epochs/windows/samples

### Filter short trials

We filter away rows where the subject didn't spend at least `min_task_duration` seconds with the task.

In [ ]:
n_prev = len(df_markers)
df_markers = df_markers[df_markers['duration'] > min_task_duration]
print(f"Filtered away {n_prev - len(df_markers)} epochs due to short duration")

### Filter no answer trials

We filter away rows where space was clicked (didn't answer/skipped/unsure?)

In [ ]:
n_prev = len(df_markers)
df_markers = df_markers[df_markers['response'].isin(['up', 'down'])]
print(f"Filtered away {n_prev - len(df_markers)} epochs due skipped by subject")

In [ ]:
if single_subject is not None:
    logger.info(f"Selecting subject {single_subject}")
    n_prev = len(df_markers)
    new_markers = df_markers[df_markers['subject'] == single_subject]
    if not new_markers.empty:
        df_markers = new_markers
    print(f"old size: {n_prev}, new size: {len(df_markers)}")
else:
    logger.info("Using all subjects")

### Exponential moving standardize

In [ ]:
if standardize:
    from braindecode.datautil import exponential_moving_standardize
    print(eeg.to_numpy())
    output = exponential_moving_standardize(eeg.to_numpy())
    print(output)
    
    for ch_idx, col in enumerate(eeg.columns):
        eeg[col] = output[:, ch_idx]
else:
    logger.info("Skipping exponential standardize")

### Bandpass filtering

 - [ ] FIXME: Why is the signal shifted after filtering? Should this be accounted for somehow?

In [ ]:
# Bandpass-filter the signal
if use_bandpass_filter:
    logger.info("Bandpass-filtering the signal")
    plot_eeg(eeg.to_numpy(), 20000, 10 * sfreq)
    
    eeg_clean = clean.filter(eeg)
    for ch_idx, col in enumerate(eeg.columns):
        eeg[col] = eeg_clean[:, ch_idx]
        
    # plot the new result
    plot_eeg(eeg.to_numpy(), 20000, 10 * sfreq)
else:
    logger.info("Skipping bandpass filtering")

### Constructing epochs

Now we match up the EEG data with the markers to create our epochs.

In [ ]:
epochs = []
for _, row in df_markers.iterrows():
    start = datetime.fromtimestamp(row['t_presented_utc'], timezone.utc)
    stop = datetime.fromtimestamp(row['t_answered_utc'], timezone.utc)
    epoch = eeg.truncate(start, stop)
    
    # Check that sample count aligns with epoch duration
    expected_samples = round(row['duration'] * sfreq)
    actual_samples = len(epoch)
    diff = expected_samples - actual_samples
    if abs(diff) > 5:
        logger.warning(f"Expected {expected_samples} samples, found {actual_samples}")
        
    epochs.append((epoch, row['type'], row['subject'], row['img']))
print(f"epochs: {len(epochs)}")

#### Epochs to windows

Now we split up the epochs into windows of a fixed size.

 - [ ] TODO: Use a sliding window approach, similar to the one in braindecode (needs care with CV)

In [ ]:
# Split epochs into windows
WINDOW_SIZE = 512

windows = []
for epoch, type, subject, img in epochs:
    for i in range(0, len(epoch), WINDOW_SIZE):
        window = epoch.iloc[i:i+WINDOW_SIZE]
        if len(window) == WINDOW_SIZE:
            windows.append((window, type, subject, img))
        else:
            logger.debug(f'epoch too small ({len(window)}), skipping')
print(f"windows: {len(windows)}")

### Constructing our `X` and `y`
 
Now to actually construct matrices that we can feed into the classifier.

In [ ]:
X, y, subj, img = zip(*windows)
X = np.array([x.to_numpy().T for x in X])
print(X.shape)

In [ ]:
# Lets take a look at the signal in a few of the windows to make sure it looks ok
fig = plt.figure(figsize=(25, 10))
axs = fig.subplots(4, 1, sharex=True, sharey=True)
for i, ax in enumerate(axs):
    plt.ylim(-40, 40)
    plt.xlim(0, WINDOW_SIZE)
    ax.plot(X[i, :, :].T);

In [ ]:
y = np.array(y)
print(y.shape)
print(Counter(y))

In [ ]:
subj = np.array(subj)
img = np.array(img)

In [ ]:
from numpy.random import shuffle

if balance_dataset:
    assert len(set(y)) == 2, 'only supports two classes'
    
    c_y: dict = Counter(y)
    c_smallest, min_count = min(c_y.items(), key=lambda t: t[1])
    print(f"Smallest class: {c_smallest} with {min_count} samples")
    
    large_class_samples = np.argwhere(y != c_smallest).flatten()
    other_class_samples = np.argwhere(y == c_smallest).flatten()
    
    # Randomly undersample the largest class
    shuffle(large_class_samples)
    subsample = large_class_samples[:min_count]
    idx = np.concatenate([subsample, other_class_samples])
    X, y, subj, img = X[idx], y[idx], subj[idx], img[idx]
    print(f"Total samples: {len(y)}")
    
    # Doesn't work as fit_resample expects X to be a 2D matrix
    #from imblearn.under_sampling import RandomUnderSampler
    #rus = RandomUnderSampler(random_state=0)
    #X_resampled, y_resampled = rus.fit_resample(X, y)
    #print(sorted(Counter(y_resampled).items()))

## Training

Here we train our model using pyRiemann.

 - [ ] Much of this code is from eegclassify/main.py, code should probably be reused better

First we set up the different classifiers we want to train:

In [ ]:
import sklearn
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import LeaveOneGroupOut, learning_curve
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

from pyriemann.estimation import Covariances, ERPCovariances, XdawnCovariances
from pyriemann.spatialfilters import CSP
from pyriemann.tangentspace import TangentSpace


# Fixes non-convergence for binary classification
dual = set(y) == 2

clfs: Dict[str, Pipeline] = {
    # These four are from https://neurotechx.github.io/eeg-notebooks/auto_examples/visual_ssvep/02r__ssvep_decoding.html
    "CSP + Cov + TS": make_pipeline(
        Covariances(),
        CSP(4, log=False),
        TangentSpace(),
        LogisticRegression(dual=dual),
    ),
    "Cov + TS": make_pipeline(
        Covariances(), TangentSpace(), LogisticRegression(dual=dual)
    ),
    "Xdawn + TS": make_pipeline(
        XdawnCovariances(2),
        TangentSpace(metric='riemann'),
        LogisticRegression()
    ),
    # Performs meh
    #"CSP + RegLDA": make_pipeline(
    #    Covariances(), CSP(4), LDA(shrinkage="auto", solver="eigen")
    #),
    # Performs badly
    # "Cov + MDM": make_pipeline(Covariances(), MDM()),
}

And then we train each classifier and plot their respective confusion matrices:

In [ ]:
from sklearn.metrics import plot_confusion_matrix
from eegclassify.util import unison_shuffled_copies
from eegclassify.main import _performance

for name, clf in clfs.items():
    logger.info(f"===== Training with {name} =====")

    # Shuffled split
    X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(
        X, y, test_size=0.3, shuffle=True
    )

    logger.info("Training...")
    clf.fit(X_train, y_train)
    logger.info(f"Test score: {clf.score(X_test, y_test)}")

    y_pred = clf.predict(X_test)
    perf = _performance(y_test, y_pred)
    logger.info(perf)
    
    disp = plot_confusion_matrix(clf, X_test, y_test,
                                 display_labels=['code', 'prose'],
                                 cmap=plt.cm.Blues,
                                 normalize='true')
    plt.show()

In [ ]:
# LORO/LOGO split
# TODO: Is LOGO the same as LORO?
logo = LeaveOneGroupOut()
groups = img

for name, clf in clfs.items():
    logger.info(f"===== Training with {name} =====")
    
    score = sklearn.model_selection.cross_val_score(clf, X, y, cv=3, n_jobs=-1)
    logger.info(f"CV score (shuffled):   {score}")
    
    # LORO
    score = sklearn.model_selection.cross_val_score(clf, X, y, cv=logo, groups=groups, n_jobs=-1)
    logger.info(f"CV score (LORO):       {np.mean(score) :.3f} (mean), {np.std(score) :.3f} (std)")
    
    # FIXME: This shouldn't run on the entire sample
    plot_confusion_matrix(clf, X, y,
                          display_labels=['code', 'prose'],
                          cmap=plt.cm.Blues,
                          normalize='true')
    plt.show()

### Learning curves

Now to check the learning curves and see if the train and validation scores converge.

**Note:** Performance is currently terrible as there isn't enough data for the model to learn to generalize across subjects (easily seen by changing to shuffled CV).

A great example of how to plot learning curves is available here: https://scikit-learn.org/stable/auto_examples/model_selection/plot_learning_curve.html

In [ ]:
from eegclassify.util import powspace

for name, clf in clfs.items():
    logger.info(f"===== Training with {name} =====")
    
    # We create shuffled versions of the dataset to ensure that all stimuli of the same type aren't in sequence
    # (as is the case for subject 1 which didn't have shuffled stimuli)
    x_l, y_l, groups_l = unison_shuffled_copies(X, y, groups)
    #x_l, y_l, groups_l = X, y, groups
    
    # We build a train_sizes to train across several sample sizes
    groups_c: dict = Counter(groups_l)
    largest_group, largest_group_count = max(groups_c.items(), key=lambda g: g[1])
    largest_loro_train = len(y) - largest_group_count
    smallest_train = 10
    
    train_sizes = np.floor(powspace(smallest_train, largest_loro_train, power=4, num=10)) / largest_loro_train
    # print(train_sizes)
    
    # Compute the learning curve
    train_sizes, train_scores, valid_scores = learning_curve(
        clf, x_l, y_l, groups=groups_l, 
        train_sizes=train_sizes, 
        cv=logo, n_jobs=-1
    )
    
    m = np.mean(train_scores, axis=1)
    std = np.std(train_scores, axis=1)
    plt.plot(train_sizes, m, label="training score", marker='.')
    plt.fill_between(train_sizes, m-std, m+std, alpha=0.2)
    
    m = np.mean(valid_scores, axis=1)
    std = np.std(valid_scores, axis=1)
    plt.plot(train_sizes, m, label="validation score", marker='.')
    plt.fill_between(train_sizes, m-std, m+std, alpha=0.2)
    
    plt.axhline(0.5, color='grey', linestyle='--', linewidth=0.8)
    plt.ylim(0, 1)
    plt.xlim(train_sizes[0], train_sizes[-1])
        
    plt.legend()
    plt.show()

# Braindecode stuff

Here we'll experiment with braindecode (convnets) to compare performance.

In [ ]:
# To use braindecode we need to transform our data to the braindecode format

from braindecode.datautil import create_from_X_y

# This wants X to be in the shape (x_trials, n_channels, n_samples)
Xb, yb, subjb, imgb = zip(*epochs)
Xb = [x.to_numpy().T for x in Xb]
print(len(Xb), Xb[0].shape)
yb = np.array([0 if yy == 'code' else 1 for yy in y])
print(yb.shape)
windows_dataset = create_from_X_y(
    Xb, yb, drop_last_window=False, sfreq=sfreq, ch_names=list(eeg.columns),
    window_stride_samples=500,
    window_size_samples=500,
)

In [ ]:
windows_dataset.description['group'] = imgb
windows_dataset.description.head(10)

In [ ]:
# Homegrown LORO
splitted = windows_dataset.split('group')
train_sets = [splitted[key] for key in list(splitted.keys())[1:]]
train_set = train_sets[0]
for ts in train_sets[1:]:
    train_sets += ts
valid_set = splitted[list(splitted.keys())[0]]

In [ ]:
import torch
from braindecode.util import set_random_seeds
from braindecode.models import ShallowFBCSPNet

cuda = torch.cuda.is_available()  # check if GPU is available, if True chooses to use it
if cuda:
    print("CUDA available!")
    torch.backends.cudnn.benchmark = True
    
# Set random seed to be able to reproduce results
seed = 20200220
set_random_seeds(seed=seed, cuda=cuda)

# Extract number of chans and time steps from dataset
n_classes = len(set(y))
n_chans = train_set[0][0].shape[0]
input_window_samples = train_set[0][0].shape[1]

print(f"classes:   {n_classes}")
print(f"channels:  {n_chans}")
print(f"samples per window:  {input_window_samples}")

model = ShallowFBCSPNet(
    n_chans,
    n_classes,
    input_window_samples=input_window_samples,
    final_conv_length='auto',
)

# Send model to GPU
if cuda:
    model.cuda()

In [ ]:
print(train_set[0])
print(train_set[0][0].shape)

In [ ]:
from skorch.callbacks import LRScheduler
from skorch.helper import predefined_split

from braindecode import EEGClassifier

# These values we found good for shallow network:
lr = 0.0625 * 0.01
weight_decay = 0

# For deep4 they should be:
# lr = 1 * 0.01
# weight_decay = 0.5 * 0.001

batch_size = 64
n_epochs = 10

clf = EEGClassifier(
    model,
    criterion=torch.nn.NLLLoss,
    optimizer=torch.optim.AdamW,
    train_split=predefined_split(valid_set),  # using valid_set for validation
    optimizer__lr=lr,
    optimizer__weight_decay=weight_decay,
    batch_size=batch_size,
    callbacks=[
        "accuracy", ("lr_scheduler", LRScheduler('CosineAnnealingLR', T_max=n_epochs - 1)),
    ],
    device='cuda' if cuda else 'cpu',
)

# Model training for a specified number of epochs. `y` is None as it is already supplied in the dataset.
# FIXME: Remove try/except when error is resolved
try:
    clf.fit(train_set, y=None, epochs=n_epochs)
except Exception as e:
    logger.exception(e)